In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import random
import os
import sklearn
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet_v3 import MobileNetV3
from keras import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
! kaggle datasets download -d chiragsoni/ferdata

In [ ]:
!unzip ferdata.zip

In [ ]:
emotions=['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
data=[]

In [ ]:
for i in emotions:
    
    path=os.path.join('/content/train',i)
    label=emotions.index(i)
    for file in os.listdir(path):
        img_path=os.path.join(path,file)
        img=cv2.imread(img_path)
        img=cv2.resize(img,(48,48))
        data.append([img,label]) 

In [ ]:
len(data)

In [ ]:
data[0]

In [ ]:
random.shuffle(data)

In [ ]:
data[0]

In [ ]:
X=[]
y=[]
for image,label in data:
    X.append(image)
    y.append(label)

In [ ]:
X

In [ ]:
X=np.array(X)

In [ ]:
X

In [ ]:
X.shape

In [ ]:
y=np.array(y)
y.shape

In [ ]:
y

In [ ]:
y[8]

In [ ]:
plt.matshow(X[8])

In [ ]:
values=np.unique(y,return_counts=True)
values

In [ ]:
values[1]

In [ ]:
plt.bar(emotions,values[1])
plt.show()

In [ ]:
data_aug_train=ImageDataGenerator(rescale=1.0/255,
                            rotation_range=30,
                            shear_range=0.2,
                            zoom_range=0.15,
                           horizontal_flip=True,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            fill_mode='nearest')

In [ ]:
data_aug_test=ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_gen=data_aug_train.flow_from_directory('/content/train',
                                       target_size=(48,48),
                                       batch_size=32,
                                       class_mode='categorical')

In [ ]:
test_gen=data_aug_test.flow_from_directory('/content/test',
                                          target_size=(48,48),
                                          batch_size=32,
                                          class_mode='categorical')

In [ ]:
model=Sequential([Conv2D(32,kernel_size=(3,3),padding='same',input_shape=(48,48,3),activation='leaky_relu'),
                 MaxPooling2D((2,2)),
                 Conv2D(64,kernel_size=(3,3),padding='same',activation='leaky_relu'),
                 MaxPooling2D((2,2)),
                 Conv2D(64,kernel_size=(3,3),padding='same',activation='leaky_relu'),
                 MaxPooling2D((2,2)),
                 Conv2D(128,kernel_size=(3,3),padding='same',activation='leaky_relu'),
                 Flatten(),
                 Dense(128,activation='leaky_relu'),
                 Dense(128,activation='leaky_relu'),
                 Dense(7,activation='softmax')
                 ])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
batch_size=32

In [ ]:
model.fit(train_gen,
          steps_per_epoch=28709//batch_size,
          validation_data=test_gen,
          validation_steps=7178//batch_size,
          epochs=100)

In [ ]:
y_pred=model.predict(test_gen,
                     steps=7178//batch_size)

In [ ]:
y_pred

In [ ]:
y_pred[0]

In [ ]:
img=cv2.imread('/content/happy1.jpg')
img=cv2.resize(img,(48,48))

In [ ]:
plt.imshow(img)

In [ ]:
img=np.reshape(img,(1,48,48,3))

In [ ]:
img

In [ ]:
img=img/255.0

In [ ]:
img

In [ ]:
model.predict(img)

In [ ]:
np.argmax(model.predict(img))

In [ ]:
emotions=['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
model.save('Model (1).h5')